# CS221 Final Project

# - Sentiment Analysis using Click Stream Data -

## DATA PROCESSING: Step 1 - Load Data

In [1]:
import pandas as pd
import os
import numpy as np
import boto3

fnameLabelled   = 'LabelledData.csv'
fnameUnLabelled = 'UnLabelledData.csv'

#read pre processed date file, read from S3 bucket otherwise
if os.path.isfile(fnameLabelled):
    print("file exists locally, loading ...")
    df_Labelled = pd.read_csv(fnameLabelled)
else:
    #read the labelled data from an AWS S3 Bucket
    print("file doesn't exists locally, loading from S3")
    s3client = boto3.client('s3')
    response = s3client.get_object(Bucket = 'click-metric-analytics', Key = 'stanford-labeled.csv')
    df_Labelled = pd.read_csv(response['Body'])
    #save locally
    df_Labelled.to_csv(fnameLabelled)
print("Loading Labelled Data Complete")

    
if os.path.isfile(fnameUnLabelled):
    print("file exists locally, loading ...")
    df_UnLabelled = pd.read_csv(fnameUnLabelled)
else:
    #Read the unlabelled data from an AWS S3 Bucket 
    print("file doesn't exists locally, loading from S3")
    s3client = boto3.client('s3')
    response = s3client.get_object(Bucket = 'click-metric-analytics', Key = 'results-query-stanford-rds.csv')
    df_UnLabelled = pd.read_csv(response['Body'])
    #save locally
    df_UnLabelled.to_csv(fnameUnLabelled)
print("Loading UnLabelled Data Complete")

file exists locally, loading ...
Loading Labelled Data Complete
file exists locally, loading ...
Loading UnLabelled Data Complete


## DATA PROCESSING: Step 2
### 2.1 Drop Corrupted Data

In [2]:
def dropCorruptedData(label, df):
    print(label + " Data Shape: Original Loaded Data", df.shape)
    #drop dataframes that start with 'clickToCustomerReady' as it only indicates a login
    #to the webpage before any activity is initiated
    list_to_drop = df.index[df['metric'] == 'clickToCustomerReady'].tolist()
    df = df.drop(df.index[list_to_drop])
    df = df.reset_index(drop=True)
    print(label + " Data Shape after dropping non relavent activities", df.shape)


    #drop dataframes with account_id == -1 (indicating an error in the captured result)
    list_to_drop = df.index[df['account_id'] == -1].tolist()
    df = df.drop(df.index[list_to_drop])
    df = df.reset_index(drop=True)
    print(label +  " Data Shape after dropping -1", df.shape)


    #drop dataframes with nan (indicating an error in the captured result)
    df = df.dropna()
    df = df.reset_index(drop=True)
    print(label +  " Data Shape after dropping NaN",df.shape)


    #drop dataframes with session_id == 1 (indicating an error in the captured result)
    list_to_drop = df.index[df['session_id'] == 1].tolist()
    df = df.drop(df.index[list_to_drop])
    df = df.reset_index(drop=True)
    print(label + " Data Shape after dropping session_id equal to 1", df.shape)
    return df


df_Labelled     = dropCorruptedData('labelled', df_Labelled)
print(" ")
print("#####################################")
print(" ")
df_UnLabelled  = dropCorruptedData('Unlabelled', df_UnLabelled)

labelled Data Shape: Original Loaded Data (35648, 6)
labelled Data Shape after dropping non relavent activities (25589, 6)
labelled Data Shape after dropping -1 (25589, 6)
labelled Data Shape after dropping NaN (25589, 6)
labelled Data Shape after dropping session_id equal to 1 (25589, 6)
 
#####################################
 
Unlabelled Data Shape: Original Loaded Data (8995093, 5)
Unlabelled Data Shape after dropping non relavent activities (5618156, 5)
Unlabelled Data Shape after dropping -1 (5617972, 5)
Unlabelled Data Shape after dropping NaN (5616783, 5)
Unlabelled Data Shape after dropping session_id equal to 1 (5616783, 5)


### 2.2 Drop Script Generated Clicks

In [3]:
#Drop Not needed Col representing index
df_Labelled = df_Labelled.drop('Unnamed: 0', 1)
df_UnLabelled = df_UnLabelled.drop('Unnamed: 0', 1)

In [4]:
# remove sessions with more clicks than max_session_activity
max_session_activity = 50
print("Shape before dropping scripted clicks:", df_UnLabelled.shape)
session_counts = pd.value_counts(df_UnLabelled['session_id'].values, sort=True)
sessions_to_remove = session_counts[session_counts >= max_session_activity]
df_UnLabelled = df_UnLabelled[~df_UnLabelled.session_id.isin(sessions_to_remove.index.tolist())]
print("Shape after dropping scripted clicks:", df_UnLabelled.shape)
print(" ")
print("#####################################")
print(" ")
print("Shape before dropping scripted clicks:", df_Labelled.shape)
session_counts = pd.value_counts(df_Labelled['session_id'].values, sort=True)
sessions_to_remove = session_counts[session_counts >= max_session_activity]
df_Labelled = df_Labelled[~df_Labelled.session_id.isin(sessions_to_remove.index.tolist())]
print("Shape after dropping scripted clicks:", df_Labelled.shape)

Shape before dropping scripted clicks: (5616783, 4)
Shape after dropping scripted clicks: (5307053, 4)
 
#####################################
 
Shape before dropping scripted clicks: (25589, 5)
Shape after dropping scripted clicks: (19413, 5)


## DATA PROCESSING: Step 3- Data Cleaning
### 3.1 Reduce name complexity

In [5]:
#Get rid of the 'click' at the beginning of the metric
df_Labelled['metric'] = [str(x)[6:] for x in df_Labelled['metric']]
df_UnLabelled['metric'] = [str(x)[6:] for x in df_UnLabelled['metric']]


#reduce the length of session_id to improve debugging visibity
df_UnLabelled_temp = df_UnLabelled
df_Labelled_temp   = df_Labelled

df_UnLabelled_temp['session_id'] = [str(x)[6:11] for x in df_UnLabelled_temp['session_id']]
df_Labelled_temp['session_id']   = [str(x)[6:11] for x in df_Labelled_temp['session_id']]
#Confirm that renaming session_id did not cause any name overlaps
assert(len(df_Labelled_temp['session_id'].unique()) == len(df_Labelled['session_id'].unique()))
assert(len(df_UnLabelled_temp['session_id'].unique()) == len(df_UnLabelled['session_id'].unique()))

df_Labelled = df_Labelled_temp
df_UnLabelled = df_UnLabelled_temp
print("Data Shape of Unlabelled Data", df_UnLabelled.shape)
print("Data Shape of labelled Data", df_Labelled.shape)

Data Shape of Unlabelled Data (5307053, 4)
Data Shape of labelled Data (19413, 5)


### 3.2 drop dataframes with only one metric per session ID activity  


In [6]:
# remove sessions with only click activity
max_session_activity = 1
print("Shape before dropping scripted clicks:", df_UnLabelled.shape)
session_counts = pd.value_counts(df_UnLabelled['session_id'].values, sort=True)
sessions_to_remove = session_counts[session_counts == max_session_activity]
df_UnLabelled = df_UnLabelled[~df_UnLabelled.session_id.isin(sessions_to_remove.index.tolist())]
print("Shape after dropping scripted clicks:", df_UnLabelled.shape)
print(" ")
print("#####################################")
print(" ")
print("Shape before dropping scripted clicks:", df_Labelled.shape)
session_counts = pd.value_counts(df_Labelled['session_id'].values, sort=True)
sessions_to_remove = session_counts[session_counts == max_session_activity]
df_Labelled = df_Labelled[~df_Labelled.session_id.isin(sessions_to_remove.index.tolist())]
print("Shape after dropping scripted clicks:", df_Labelled.shape)

Shape before dropping scripted clicks: (5307053, 4)
Shape after dropping scripted clicks: (5163748, 4)
 
#####################################
 
Shape before dropping scripted clicks: (19413, 5)
Shape after dropping scripted clicks: (19040, 5)


## DATA PROCESSING: Step 4
### Generate Batches of Unlabelled Data for the Self-Learning Semi Supervised Function

In [8]:
df_UnLabelled =  df_UnLabelled.sort_values(by=['session_id'])
session_id_list = df_UnLabelled['session_id'].unique()
listToPickFrom = np.random.permutation(len(session_id_list))

In [9]:
dataFrameList = []
number_of_batches = 100
batch_size = 250

for batch_index in range(0, number_of_batches):
    listID = session_id_list[listToPickFrom[batch_index*batch_size: batch_size*(batch_index+1)]]
    dataFrameList.append(df_UnLabelled.loc[df_UnLabelled['session_id'].isin(listID)])

## DATA PROCESSING: Step 5
### Save results in a csv file

In [10]:
#save filteres labelled data into a csv file
fnameLabelledProcessed = 'fnameLabelledProcessed.csv'
df_Labelled.to_csv(fnameLabelledProcessed)

writer = pd.ExcelWriter('UnlabelledBatch.xlsx', engine='xlsxwriter')
for batch_index in range(len(dataFrameList)):
    fnameLabelledProcessed = 'fnameUnLabelledBatches-{}.csv'.format(batch_index)
    dataFrameList[batch_index].to_excel(writer, sheet_name=fnameLabelledProcessed)   
    
writer.save()

In [11]:
x = 0
y = 99999999
for i in range(0, 100):
    x= max(x, len(dataFrameList[i]))
    y= min(y, len(dataFrameList[i]))
    
print(x,y)

1588 1169
